In [1]:
import sys
sys.path.append('../..')
from src.models import *

%load_ext autoreload
%autoreload 2

%load_ext tensorboard
%tensorboard --logdir '../../logs' --bind_all

In [ ]:
%run -i 'train_MLP.py'

In [ ]:
%run -i 'train_GMF.py'

In [ ]:
%run -i 'train_both.py'